In [1]:
#First step is to demultiplex each of the 5 runs. Will use AMPtk to demultiplex the data
#this will keep only amplicons where both forward and reverse primers are found, as well as discard reads < 125 bp
!for i in MHC2a MHC2b MHC3 MHC4 MHC5; \
    do amptk ion -i ../$i.fastq -m ../$i.mapping_file.txt -o $i --full_length --min_len 125; \
    done

-------------------------------------------------------
[03:24:07 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[03:24:08 PM]: amptk v.0.9.3, USEARCH v9.2.64, VSEARCH v2.4.2
[03:24:08 PM]: Foward primer: AGGGTGCTCCTCACAGCCCTGTG,  Rev comp'd rev primer: CAGGAAATKYGCYGCGGGGACA
[03:24:08 PM]: Loading FASTQ Records
[03:24:12 PM]: 2,840,048 reads (1.7 GB)
-------------------------------------------------------
[03:54:50 PM]: Concatenating Demuxed Files
[03:54:54 PM]: 2,840,048 total reads
[03:54:54 PM]: 2,517,886 valid Barcode
[03:54:54 PM]: 2,321,440 Fwd Primer found, 326,493 Rev Primer found
[03:54:54 PM]: 5,127 discarded too short (< 125 bp)
[03:54:54 PM]: 321,366 valid output reads
[03:54:55 PM]: Found 44 barcoded samples
                Sample:  Count
                 NY040:  26088
                 NY012:  25348
                 NY047:  21492
                 WI091:  15646
                 NY001:  13602
                 NY017:  13353
                 NY008:  12801
     

In [2]:
#concatenate all files together
!cat mhc2a.demux.fq mhc2b.demux.fq mhc3.demux.fq mhc4.demux.fq mhc5.demux.fq > mhc.all.demux.fq

In [3]:
!amptk show -i mhc.all.demux.fq

----------------------------------
Found 181 barcoded samples
              Sample:  Count
               NY032:  51161
               NY040:  27597
               NY012:  27376
               NY047:  25972
               NY053:  21419
               WI107:  20334
               WI091:  18855
               PA012:  17703
               WI098:  16590
               NY001:  16078
               VT006:  15094
               WI054:  14924
               NY017:  14741
               WI105:  14021
               NY008:  13976
               NY016:  13900
               PA001:  13886
               MI130:  13828
              NY033B:  13729
               WI169:  13544
               WI076:  13365
                NY34:  13348
               VT018:  13180
               NY051:  13158
               VT016:  12810
               NY004:  12751
               WI078:  12562
               WI062:  12531
               WI166:  12342
               NY014:  12063
               NY048:  11688
          

In [4]:
#pull out the small subset for methods, as they can't be used in larger study
!amptk select -i mhc.all.demux.fq -f ../mhc.subset.methods.txt -o mhc.methods.demux.fq

Kept 74363 reads out of 1178970 total reads


In [5]:
!amptk show -i mhc.methods.demux.fq

----------------------------------
Found 17 barcoded samples
              Sample:  Count
               WI169:  13544
               WI166:  12342
               WI171:  9252
               WI168:  8124
               WI163:  5717
               IL030:  4771
               IL020:  4668
               WI165:  3531
               IL032:  3267
               IL024:  2894
               WI170:  1578
               IL022:  1553
               WI172:  1387
               WI167:  1226
               WI164:  264
               IL016:  171
               IL018:  74
----------------------------------
Read count: 74363
Read length average: 305
Read length range: 125 - 497 bp
----------------------------------


In [51]:
#now run through MHC DADA2 script, since only a few samples, don't remove alleles found only 1
#try increase quality threshold to 2.0, since we have longer reads and need the 3' end 
#to serve as an anchor, i.e. that is where translation starts
!mhc-dada2.py -i mhc.methods.demux.fq --filter_allele 1  \
              -e 2.0 --rev_comp -m ../mhc.mapping_file.txt -o methods --debug

-------------------------------------------------------
[05:30:04 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[05:30:04 PM]: Reverse complementing FASTQ records
[05:30:13 PM]: Loading FASTQ Records
[05:30:13 PM]: 74,363 reads (45.2 MB)
[05:30:13 PM]: Quality Filtering, expected errors < 2.0
[05:30:14 PM]: 47,638 reads passed
[05:30:14 PM]: Splitting FASTQ file by Sample
[05:30:17 PM]: Removed 6 samples with read number < 500 (--min_reads_sample)
[05:30:17 PM]: Running DADA2 pipeline
[05:31:50 PM]: R v3.3.3, DADA2 v1.3.5
[05:31:50 PM]: 306 total inferred sequences (iSeqs)
[05:31:50 PM]: 27 denovo chimeras removed
[05:31:50 PM]: 279 valid iSeqs
[05:31:50 PM]: Translating to Amino Acid Sequence in 3 frames
[05:31:50 PM]: 111 total translated sequences > 75 aa
[05:31:50 PM]: Running HMMER3 MHC HMM model
[05:31:50 PM]: 87 pass HMM filter, 84 are unique AA sequence.
[05:31:50 PM]: Mapping reads to DADA2 filtered iSeqs
[05:32:04 PM]: 48,848 reads mapped to iSeqs (66%)
[05:32

In [52]:
!amptk heatmap -i methods.otu_table.txt -o methods.pdf -d tsv -m heatmap

In [46]:
#run phyloseq/vegan script
!$HOME/amptk/util/phyloseq_graphs.R methods.biom methods.tree.phy methods

Loading required package: phyloseq
Loading required package: methods
Loading required package: ggplot2
Loading required package: plyr
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.4-3
[1] ‘1.19.1’
[1] ‘2.2.1’
[1] ‘1.8.4’
[1] ‘2.4.3’
Run 0 stress 0.1461771 
Run 1 stress 0.1904085 
Run 2 stress 0.1461772 
... Procrustes: rmse 0.0001741912  max resid 0.0003391395 
... Similar to previous best
Run 3 stress 0.1488007 
Run 4 stress 0.146177 
... New best solution
... Procrustes: rmse 0.0001888352  max resid 0.0004258482 
... Similar to previous best
Run 5 stress 0.1461771 
... Procrustes: rmse 9.258806e-05  max resid 0.0001961878 
... Similar to previous best
Run 6 stress 0.1904087 
Run 7 stress 0.1461771 
... Procrustes: rmse 0.0001820524  max resid 0.0003492834 
... Similar to previous best
Run 8 stress 0.146177 
... New best solution
... Procrustes: rmse 6.788699e-05  max resid 0.0001196706 
... Similar to previous best

![Heatmap](Methods.pdf)

In [17]:
#now try DADA2 workflow with the rest of the data, drop the Illinois samples as small dataset and meta data is
#questionable as well as Neda mine as we don't have any qPCR data for those sample
#these samples were used for a smaller subset for methods manuscript
!amptk remove -i mhc.all.demux.fq -f ../mhc.subset.methods.txt -o mhc.large.demux.fq

Removed 17 samples
Kept 1104607 reads out of 1178970 total reads


In [53]:
#now run through DADA2
!mhc-dada2.py -i mhc.large.demux.fq -e 2.0 --rev_comp -m ../mhc.mapping_file.txt -o all 

-------------------------------------------------------
[05:44:00 PM]: OS: MacOSX 10.12.4, 8 cores, ~ 16 GB RAM. Python: 2.7.12
[05:44:00 PM]: Reverse complementing FASTQ records
[05:46:03 PM]: Loading FASTQ Records
[05:46:04 PM]: 1,104,607 reads (639.3 MB)
[05:46:04 PM]: Quality Filtering, expected errors < 2.0
[05:46:18 PM]: 861,929 reads passed
[05:46:18 PM]: Splitting FASTQ file by Sample
[05:47:22 PM]: Removed 27 samples with read number < 500 (--min_reads_sample)
[05:47:22 PM]: Running DADA2 pipeline
[09:13:26 PM]: R v3.3.3, DADA2 v1.3.5
[09:13:26 PM]: 1,758 total inferred sequences (iSeqs)
[09:13:26 PM]: 120 denovo chimeras removed
[09:13:26 PM]: 1,638 valid iSeqs
[09:13:26 PM]: Translating to Amino Acid Sequence in 3 frames
[09:13:26 PM]: 657 total translated sequences > 75 aa
[09:13:26 PM]: Running HMMER3 MHC HMM model
[09:13:27 PM]: 483 pass HMM filter, 436 are unique AA sequence.
[09:13:27 PM]: Mapping reads to DADA2 filtered iSeqs
[09:17:05 PM]: 917,765 reads mapped to iSeq

In [55]:
!$HOME/amptk/util/phyloseq_graphs.R all.update.biom all.tree.phy all

Loading required package: phyloseq
Loading required package: methods
Loading required package: ggplot2
Loading required package: plyr
Loading required package: vegan
Loading required package: permute
Loading required package: lattice
This is vegan 2.4-3
[1] ‘1.19.1’
[1] ‘2.2.1’
[1] ‘1.8.4’
[1] ‘2.4.3’
Run 0 stress 0.2526258 
Run 1 stress 0.2604341 
Run 2 stress 0.2643054 
Run 3 stress 0.2577333 
Run 4 stress 0.2585463 
Run 5 stress 0.2562278 
Run 6 stress 0.2554188 
Run 7 stress 0.2526192 
... New best solution
... Procrustes: rmse 0.001918925  max resid 0.02036672 
Run 8 stress 0.2562992 
Run 9 stress 0.2616752 
Run 10 stress 0.2526209 
... Procrustes: rmse 0.007176689  max resid 0.07373238 
Run 11 stress 0.2567007 
Run 12 stress 0.2547139 
Run 13 stress 0.2526505 
... Procrustes: rmse 0.007363126  max resid 0.07308424 
Run 14 stress 0.2525594 
... New best solution
... Procrustes: rmse 0.004426885  max resid 0.04305596 
Run 15 stress 0.253885 
Run 16 stress 0.2594483 
Run 17 stress 0

![SiteMap](phyloseq_all/all.location.raup.pdf)

In [57]:
#### There seems to be very little to no structure in these data, even between Eastcoast (NY,VT,PA) vs Midwest (WI,MI).  
Call:
adonis(formula = rp ~ metadata[[location]], permutations = 9999) 

Permutation: free
Number of permutations: 9999

Terms added sequentially (first to last)

                           Df SumsOfSqs MeanSqs F.Model      R2 Pr(>F)
metadata[location]]   1     0.364 0.36404  1.7046 0.01314 0.2487
Residuals                 128    27.335 0.21356         0.98686       
Total                     129    27.700                 1.00000       

#### There is also nothing significant when we look at WNS positive vs WNS negative
Call:
adonis(formula = rp ~ metadata[CT_result], permutations = 9999) 

Permutation: free
Number of permutations: 9999

Terms added sequentially (first to last)

                           Df SumsOfSqs MeanSqs F.Model      R2 Pr(>F)
metadata[CT_result]]   3    0.5229 0.17429 0.80806 0.01888    0.6
Residuals                 126   27.1767 0.21569         0.98112       
Total                     129   27.6996                 1.00000       


SyntaxError: invalid syntax (<ipython-input-57-fa43aa295d37>, line 2)

![SiteMap](phyloseq_all/all.CT_result.raup.pdf)

#### Of course Michelle you can look at these data in a lot more detail than I am able to, but if there are not even site differences or geographic differences between populations - there is perhaps no chance of being any correlation with WNS/Pd.  If that is the case, not sure how we want to proceed with these data....